In [1]:
from ALLCools.mcds import MCDS
import pandas as pd
import dask
import pathlib
from wmb import brain, aibs, broad, mm10

In [2]:
metadata_path = '../221027_AMB_metadata.csv'
metadata = pd.read_csv(metadata_path, index_col=0)

gene_meta = pd.read_csv("../GeneMetadata.csv.gz", index_col=0)
gene_meta['length'] = gene_meta['end'] - gene_meta['start']

In [3]:
use_genes = gene_meta[gene_meta['length'] > 1000].index

In [4]:
n_cell = 5
dataset = 'AIBS_TENX'

## gen mc adata

In [3]:
mcds_paths = '../230410_panel_design_v1/geneslop2k_frac.mcds'

obs_dim = 'cell'
var_dim = 'geneslop2k'

with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    # still use all the cells to load MCDS
    total_mcds = MCDS.open(mcds_paths,
                           obs_dim=obs_dim,
                           use_obs=metadata.index)

In [4]:
mch_adata = total_mcds.get_adata(mc_type='CHN',
                           var_dim=var_dim,
                           select_hvf=False)


In [5]:
mch_adata.X = - mch_adata.X

In [6]:
pathlib.Path('adata').mkdir(exist_ok = True)
mch_adata.write_h5ad(f'./adata/All_Genes.mCH.h5ad')

mch_adata

AnnData object with n_obs × n_vars = 132747 × 39681
    var: 'chrom', 'end', 'start'

## gen rna adata

In [5]:
rna_annot = aibs.get_tenx_annot()

In [6]:
cells = pd.Series(rna_annot.get_index('cell'))
cells = cells.sample(100000, random_state=0)
len(cells)

100000

In [7]:
if dataset == 'AIBS_SMART':
    mcds_path = aibs.AIBS_SMART_ZARR_PATH
    gene_map = aibs.get_smart_gene_map()
elif dataset == 'AIBS_TENX':
    mcds_path = aibs.AIBS_TENX_ZARR_PATH
    gene_map = aibs.get_tenx_gene_map()
else:
    mcds_path = broad.BROAD_TENX_ZARR_PATH
    gene_map = broad.get_tenx_gene_map()

In [8]:
rna_ds = MCDS.open(mcds_path, use_obs = cells, var_dim='gene')

In [9]:
adata = rna_ds.get_count_adata(da_name='gene_da', use_vars=use_genes)

KeyError: "not all values found in index 'gene'"